<a href="https://colab.research.google.com/github/Jackuna/ai/blob/main/langchain_with_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community langchainhub chromadb langchain langchain-huggingface

In [ ]:
from google.colab import userdata
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(web_paths=["https://economictimes.indiatimes.com/news/new-updates/tcs-new-bench-policy-a-new-rule-limits-the-bench-period-to-35-days-per-year-here-is-what-it-means/articleshow/121899344.cms"])
docs = loader.load()
print(docs)

[Document(metadata={'source': 'https://economictimes.indiatimes.com/news/new-updates/tcs-new-bench-policy-a-new-rule-limits-the-bench-period-to-35-days-per-year-here-is-what-it-means/articleshow/121899344.cms', 'title': 'TCS New Bench Policy: A new rule limits the bench period to 35 days per year. Here is what it means - The Economic Times', 'description': 'Tata Consultancy Services introduces a new policy. It requires employees to be billed for at least 225 days annually. This reduces bench time to 35 days. The policy impacts compensation and career growth. Employees must upskill during unallocated periods. Work from office is now the default. Short-term project assignments are discouraged. The policy aims for better workforce utilization.', 'language': 'en'}, page_content='TCS New Bench Policy: A new rule limits the bench period to 35 days per year. Here is what it means - The Economic Times  Benchmarks Nifty24,868.95-77.55FEATURED FUNDS★★★★★HSBC Large Cap Fund Direct-Growth5Y Return

In [ ]:
!pip install -qU langchain-text-splitters

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(docs)

In [ ]:
print(str(len(texts)))

48


In [ ]:
print(texts[6].page_content)


It requires employees to be billed for at least 225 days annually. This reduces bench time to 35 days. The policy impacts compensation and career growth. Employees must upskill during unallocated periods. Work from office is now the default. Short-term project assignments are discouraged. The policy aims for better workforce utilization.AgenciesTCS bench policy Tata Consultancy Services (TCS) has rolled out a new associate deployment policy that mandates a minimum of 225 billed business days


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=texts, embedding=HuggingFaceEmbeddings())



<ipython-input-90-1946496588>:4: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = Chroma.from_documents(documents=texts, embedding=HuggingFaceEmbeddings())


In [ ]:
print(vectorstore._collection.count())
print(vectorstore._collection.get())


133
{'ids': ['5ca5717f-bf81-4326-99b9-6bec396fc0fc', 'f68a4dd3-6602-420f-8da5-0af4e633bf5f', 'a8d99cc3-e91f-44cb-bc40-322fa25c3975', 'd445f5e6-32b3-4728-b06e-2e199f882f6e', '4464c717-1bf3-4021-85fa-6c13f048fc52', 'af84dc22-b38e-4c70-93fe-0bb64d6c0008', '49c6d48a-7d9f-493b-b7e7-1d92b46a847c', '874dce6b-269f-4087-85e8-b1905f8a5311', '8a21a412-edf0-417e-85e6-cc00b502d4cc', 'ea60e356-9944-4942-8fdf-ddb038114ff8', '96cd7067-a152-4e45-950c-290e1c50361e', '8bf1a913-8825-458e-8089-08609823c7d5', '8826e8d8-b167-4649-b31e-f281cebaedde', '42477759-a68b-4918-85db-a97a8d13c0a7', 'fce34033-17d8-4d40-820d-efa0f09c90bd', '413515df-fc95-482c-bef0-27c4b2ff3498', '5da7e750-677c-4445-a6db-7a81a17c0459', 'a316d4ca-e1b8-438e-a709-973a4ef2ad6e', '254eeb3c-5bd6-4822-b31f-227045ba42f7', 'd699fd53-7bb6-4dd9-aa0f-07d530820368', '6159c3cb-f236-4c18-bf0c-ee415f6c5dce', '6d23b8c5-8a10-4148-8c1f-f0593df09913', 'ac9a59ce-bf59-4aa0-be73-ac9648124d84', 'b55ff3bb-f7cb-4903-a80a-299c9d258b45', 'f063f089-669d-4f16-a313-d5

In [ ]:
print(vectorstore.similarity_search("bench", k=2))

[Document(metadata={'source': 'https://economictimes.indiatimes.com/news/new-updates/tcs-new-bench-policy-a-new-rule-limits-the-bench-period-to-35-days-per-year-here-is-what-it-means/articleshow/121899344.cms', 'language': 'en', 'description': 'Tata Consultancy Services introduces a new policy. It requires employees to be billed for at least 225 days annually. This reduces bench time to 35 days. The policy impacts compensation and career growth. Employees must upskill during unallocated periods. Work from office is now the default. Short-term project assignments are discouraged. The policy aims for better workforce utilization.', 'title': 'TCS New Bench Policy: A new rule limits the bench period to 35 days per year. Here is what it means - The Economic Times'}, page_content='currently reporting to RMG.Live EventsUpskilling mandatory for those on benchWhile on the bench, associates are expected to dedicate 4 to 6 hours daily for learning through internal platforms such as iEvolve, Fresc

In [ ]:
retreiver = vectorstore.as_retriever()

In [ ]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
!pip install langchain_huggingface

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
  return "\n".join(doc.page_content for doc in docs)


In [ ]:
rag_chain = ({'context': retreiver | format_docs, 'question': RunnablePassthrough()}
             | prompt
             | llm
             | StrOutputParser())

In [ ]:
rag_chain.invoke("How many days employee has to spend in project ? ")

" According to TCS's updated associate deployment policy, employees are required to spend at least 225 billed business days on a project in the last 12 months to avoid negatively impacting their compensation, career growth, overseas deployment opportunities, and continuity of employment with the company. Remaining unallocated for extended periods is discouraged. Short-term project assignments are also discouraged under this policy, which aims for better workforce utilization."